# Imports des modules

In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import plotly.express as px
import streamlit as st
import time

# Chargement des données

## Fonction pour lire et traiter les fichiers *title_akas* et *title_basics*

In [2]:
#@st.cache_data
def load_and_process_title_akas_and_basics():
    # Définition des colonnes à conserver lors de la lecture du fichier csv title.akas
    columns_to_include_akas = ['titleId', 'title', 'region']
    
    # Lecture du fichier csv
    df_title_akas_pyarrow_trim_dtype = pd.read_csv(
        r"https://datasets.imdbws.com/title.akas.tsv.gz", usecols = columns_to_include_akas,
        dtype = {'titleId': 'string', 'title': 'string', 'region': 'string'}, delimiter = '\t', engine = "pyarrow")
    
    # Filtre sur la colonne "region" pour ne garder que la valeur "FR"
    df_title_akas_fr = df_title_akas_pyarrow_trim_dtype.loc[df_title_akas_pyarrow_trim_dtype["region"] == "FR"]
    
    # Suppression de la colonne "region"
    df_title_akas_fr_trim = df_title_akas_fr.drop(columns = "region")
    
    # Définition des colonnes à conserver lors de la lecture du fichier csv title.basics
    columns_to_include_basics = ['tconst', 'titleType', 'startYear', 'runtimeMinutes', 'genres']
    
    # Initialisation d'un DataFrame pour la lecture du fichier title.basics
    df_title_basics_recent_years = pd.DataFrame()
    
    # Définition des "chunks"
    chunksize = 500000
    df_chunks = pd.read_csv(
        r"https://datasets.imdbws.com/title.basics.tsv.gz", usecols = columns_to_include_basics,
        dtype = {'tconst': 'string', 'titleType': 'string', 'startYear': 'string',
                 'runtimeMinutes': 'string', 'genres': 'string'},
        delimiter = '\t', low_memory = False, chunksize = chunksize)

    for chunk in df_chunks:
        # Filtrer et traiter chaque morceau
        
        # Suppression des lignes pour lesquelles la colonne "startYear" ne contient pas de nombre
        chunk = chunk[chunk["startYear"].str.isnumeric()]
        
        # Changement de type de données de la colonne "startYear", passage en type "integer"
        chunk["startYear"] = chunk["startYear"].astype("int32")
        
        # Filtre sur la colonne "startYear" pour ne garder que les années à partir de 1980
        chunk = chunk[chunk["startYear"] >= 1980]
        
        # Filtre sur la colonne "titleType" pour ne garder les titres de type "movie"
        chunk = chunk[chunk["titleType"] == "movie"]
        
        # Concaténer le morceau traité au DataFrame final
        df_title_basics_recent_years = pd.concat([df_title_basics_recent_years, chunk])
    
    # Fusion des deux DataFrame
    df_title_fr_recent_years = pd.merge(
        left = df_title_basics_recent_years, right = df_title_akas_fr_trim, how = 'inner',
        left_on = "tconst", right_on = "titleId")
    
    # Suppression des doublons
    df_movie_fr_recent_years_trim = df_title_fr_recent_years.copy()
    df_movie_fr_recent_years_trim.drop_duplicates(subset = 'tconst', inplace = True)
    
    # Suppression des colonnes inutiles
    # Colonne "titleType" devenue inutile car contient uniquement "movie"
    df_movie_fr_recent_years_trim.drop(columns = "titleType", inplace = True)
    # Colonne "titleId" car en doublon avec "tconst"
    df_movie_fr_recent_years_trim.drop(columns = "titleId", inplace = True)
    
    # Suppression des lignes où il n'y a pas de genre ("\N")
    df_movie_fr_recent_years_trim = df_movie_fr_recent_years_trim[
        df_movie_fr_recent_years_trim["genres"] != "\\N"]
    
    return df_movie_fr_recent_years_trim

start_time = time.time()
df_movie_fr_recent_years_trim = load_and_process_title_akas_and_basics()
end_time = time.time()
loading_time = end_time - start_time
print(f"Temps de chargement : {loading_time} secondes")

Temps de chargement : 136.13365745544434 secondes


In [3]:
df_movie_fr_recent_years_trim.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48862 entries, 0 to 54740
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tconst          48862 non-null  string
 1   startYear       48862 non-null  int32 
 2   runtimeMinutes  48862 non-null  string
 3   genres          48862 non-null  string
 4   title           48862 non-null  string
dtypes: int32(1), string(4)
memory usage: 2.1 MB


# Traitement des genres

## Extraction des genres avec leur nombre d'occurences, suppression des genres les moins représentés

In [4]:
#@st.cache_data
def process_genres(df):
    # Création d'une copie du DataFrame et reset des index
    df_copy = df.copy()
    df_copy.reset_index(inplace = True, drop = True)
    df_copy["genres"] = df_copy["genres"].astype("object")

    # Initialisation d'un dictionnaire qui va servir à lister les genres présents, avec :
        # chaque clé est un genre
        # chaque valeur est la nombre d'occurences de ce genre
    dict_genres = {}

    list_tmp = [] # liste temporaire utilisée dans la boucle
    nb_movies = len(df_copy) # Nbre de films

    # Boucle sur les films de notre DataFrame ..._copy
    for i in range(nb_movies):
        list_tmp = df_copy["genres"][i].split(",")
            # Affectation la liste des genres de la ligne courant à la ligne temp
        df_copy["genres"][i] = list_tmp
            # Remplacement de la valeur dans la colonne "genres" par cette liste de genres
        # Vérification, pour chaque genre de cette liste, s'il est déjà présent dans le dictionnaire
        for str_genre in list_tmp:
            if not str_genre in dict_genres:
                # Si le genre n'est pas dans le dictionnaire, on l'ajoute
                dict_genres[str_genre] = 1
            else:
                # Si le genre est déjà dans le dictionnaire, on incrémente son nombre d'occurences
                dict_genres[str_genre] += 1
    
    # Création d'un DataFrame des genres
    df_genres = pd.DataFrame.from_dict(dict_genres, orient = 'index', columns = ['Occurences'])
    df_genres.sort_values(by = "Occurences", ascending = False, inplace = True)
    df_genres.reset_index(inplace = True)
    df_genres.rename(columns = {"index" : "Genre"}, inplace = True)
    
    # Liste des genres que l'on veut supprimer
    list_genres_to_delete = ['Adult','News', 'Reality-TV', 'Talk-Show', 'Short', 'Game-Show']

    # Ajout d'une colonne "genres_to_delete" montrant s'il y a l'un des genres à supprimer dans la colonne "genres" (True)
    df_copy["genres_to_delete"] = df_copy.genres.apply(lambda x : any(gen in x for gen in list_genres_to_delete))
    
    # Suppression des lignes pour lesquelles "genres_to_delete" est True, c'est-à-dire
    # les lignes ayant un genre que l'on veut supprimer
    df_copy.drop(df_copy[df_copy.genres_to_delete == True].index, inplace = True)
    
    # Reset des index
    df_copy.reset_index(inplace = True, drop = True)
    
    # Suppression de la colonne temporaire "genres_to_delete"
    df_copy.drop("genres_to_delete", axis = 1, inplace = True)
    
    return df_copy, df_genres

df_movie_fr_recent_years_trim_copy, df_genres = process_genres(df_movie_fr_recent_years_trim)

C:\Users\julie\AppData\Local\Temp\ipykernel_78744\1461944247.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy["genres"][i] = list_tmp


In [5]:
df_movie_fr_recent_years_trim_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47932 entries, 0 to 47931
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tconst          47932 non-null  string
 1   startYear       47932 non-null  int32 
 2   runtimeMinutes  47932 non-null  string
 3   genres          47932 non-null  object
 4   title           47932 non-null  string
dtypes: int32(1), object(1), string(3)
memory usage: 1.6+ MB


In [6]:
df_genres["Selected"] = False
df_genres

,Genre,Occurences,Selected
0,Drama,22181,False
1,Comedy,11764,False
2,Documentary,10236,False
3,Action,6561,False
4,Thriller,5772,False
5,Crime,5185,False
6,Romance,5179,False
7,Adventure,4163,False
8,Horror,4103,False
9,Mystery,2485,False


## "Eclatement" de la colonne "genres"

In [7]:
# Création d'un DataFrame avec la colonne "genres" éclatée
df_movie_title_fr_recent_years_exploded = df_movie_fr_recent_years_trim_copy.explode(column = "genres")
df_movie_title_fr_recent_years_exploded["genres"] = df_movie_title_fr_recent_years_exploded["genres"].astype("string")
df_movie_title_fr_recent_years_exploded.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92074 entries, 0 to 47931
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tconst          92074 non-null  string
 1   startYear       92074 non-null  int32 
 2   runtimeMinutes  92074 non-null  string
 3   genres          92074 non-null  string
 4   title           92074 non-null  string
dtypes: int32(1), string(4)
memory usage: 3.9 MB


## Réduction du nombre de genres

In [8]:
# Définition d'une liste de genres à conserver
#list_genres_to_keep = ['Drama', 'Comedy', 'Documentary', 'Action', 'Thriller', 'Crime', 'Romance', 'Adventure', 'Horror',
#                       'Mystery', 'Fantasy', 'Biography', 'Family', 'Sci-Fi', 'Animation', 'History', 'Music', 'War',
#                       'Sport', 'Musical', 'Western']
list_genres_to_keep = ['Drama', 'Comedy', 'Action', 'Thriller', 'Crime',
                       'Romance', 'Adventure', 'Horror', 'Mystery', 'Fantasy']

df_genres.loc[df_genres["Genre"].isin(list_genres_to_keep), "Selected"] = True

# Création d'un DataFrame des genres ne contenant que les genres à conserver
#df_genres_trim = df_genres.loc[df_genres.Genre.isin(list_genres_to_keep)]
df_genres_trim = df_genres.loc[df_genres.Selected]
df_genres_trim

,Genre,Occurences,Selected
0,Drama,22181,True
1,Comedy,11764,True
3,Action,6561,True
4,Thriller,5772,True
5,Crime,5185,True
6,Romance,5179,True
7,Adventure,4163,True
8,Horror,4103,True
9,Mystery,2485,True
11,Fantasy,2119,True


In [9]:
# Création d'un DataFrame avec uniquement les genres à conserver définis plus haut
df_movie_title_fr_recent_years_exploded_trim = df_movie_title_fr_recent_years_exploded.loc[
    df_movie_title_fr_recent_years_exploded["genres"].isin(df_genres_trim["Genre"])]

df_movie_title_fr_recent_years_exploded_trim.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69272 entries, 1 to 47931
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tconst          69272 non-null  string
 1   startYear       69272 non-null  int32 
 2   runtimeMinutes  69272 non-null  string
 3   genres          69272 non-null  string
 4   title           69272 non-null  string
dtypes: int32(1), string(4)
memory usage: 2.9 MB


# Fusion des données avec les notes et votes ("*title_ratings*")

## Fonction pour lire et traiter les fichiers *title_ratings*

In [10]:
#@st.cache_data
def load_and_process_title_ratings():
    df_title_ratings = pd.read_csv(r"https://datasets.imdbws.com/title.ratings.tsv.gz",
                                   delimiter = '\t', low_memory = False)
    return df_title_ratings

df_title_ratings = load_and_process_title_ratings()

## Fusion avec le DataFrame des films par genre éclaté

In [11]:
# Création d'un nouveau DataFrame comme résultat de cette jointure
# Merge de type inner join (how = 'inner') sur les champs "tconst" de chaque table
# pour récupérer les lignes pour lesquelles les notes et nombes de votes sont disponibles.
df_movie_title_fr_recent_years_exploded_ratings = pd.merge(
    left = df_movie_title_fr_recent_years_exploded_trim, right = df_title_ratings, how = 'inner',
    left_on = "tconst", right_on = "tconst")

## Nouveau DataFrame pour aggréger les données par genre et par année

In [12]:
# Création d'un nouveau DataFrame pour ne conserver que les colonnes qui nous intéressent
df_years_genres_ratings = df_movie_title_fr_recent_years_exploded_ratings[["startYear", "genres", "averageRating", "numVotes"]]

# Nouvelle colonne "weighted_rating" pour le calcul des moyennes pondérées
df_years_genres_ratings["weighted_rating"] = df_years_genres_ratings.averageRating * df_years_genres_ratings.numVotes

# Création d'un nouveau DataFrame df_group_years_genres en groupant les données du précédent par année et par genre
# avec comme aggrégation les somme des nombres de votes et des valeurs de la nouvelle colonne weighted_rating"
df_group_years_genres = df_years_genres_ratings.groupby(
    by = ["startYear", "genres"]).agg({"numVotes" : "sum", "weighted_rating" : "sum"})

# Division de la colonne "weighted_rating"  par le nombre de vote pour calculer la moyenne pondérée
df_group_years_genres.weighted_rating = df_group_years_genres.weighted_rating / df_group_years_genres.numVotes

# Création d'un DataFrame pour le tracé, en ré-initialisant les index
df_group_years_genres_to_plot = df_group_years_genres.reset_index()

C:\Users\julie\AppData\Local\Temp\ipykernel_78744\436460260.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_years_genres_ratings["weighted_rating"] = df_years_genres_ratings.averageRating * df_years_genres_ratings.numVotes


In [13]:
df_group_years_genres_to_plot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440 entries, 0 to 439
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   startYear        440 non-null    int64  
 1   genres           440 non-null    string 
 2   numVotes         440 non-null    int64  
 3   weighted_rating  440 non-null    float64
dtypes: float64(1), int64(2), string(1)
memory usage: 13.9 KB


In [14]:
df_group_years_genres_to_plot

,startYear,genres,numVotes,weighted_rating
0,1980,Action,1928345,8.114136
1,1980,Adventure,2129972,7.934840
2,1980,Comedy,1170877,7.088511
3,1980,Crime,315144,6.661269
4,1980,Drama,2523690,7.809570
...,...,...,...,...
435,2023,Fantasy,51166,6.158119
436,2023,Horror,361767,6.218669
437,2023,Mystery,311774,6.424598
438,2023,Romance,168467,5.951151


# Tracés

## Bubble plot de la moyenne pondérée (y) par année (x) et par genre (catégorie), avec le nombre de votes pour la taille des bulles

In [17]:
#color_list = ['rgb(204, 0, 0)', 'rgb(153, 0, 0)', 'rgb(102, 0, 0)', 'rgb(51, 0, 0)',
#              'rgb(204, 204, 0)', 'rgb(153, 153, 0)', 'rgb(102, 102, 0)', 'rgb(51, 51, 0)']
fig = px.scatter(data_frame = df_group_years_genres_to_plot, x = "startYear", y = "weighted_rating",
                 size = "numVotes", color = "genres", color_discrete_sequence = px.colors.qualitative.Light24,
                 width = 1000, height = 600,
                 labels = {"startYear": "Year", "weighted_rating": "Weighted rating", "genres": "Movie genre",
                          "numVotes": "Number of votes"},
                 title = "Ratings and number of votes of movies produced per year and genre")

fig.update_layout(title = {'y': 0.9, 'x': 0.5, 'xanchor': 'center', 'yanchor': 'top',
                           'font' : dict(size = 24)},
                 plot_bgcolor = 'white')

fig.update_xaxes(ticks = 'outside', showline = True, linecolor = 'black', linewidth = 2,
                 gridcolor = 'lightgrey', griddash = 'dash',
                 range=[1979, 2023])
fig.update_yaxes(ticks = 'outside', showline = True, linecolor = 'black', linewidth = 2,
                 gridcolor = 'lightgrey', griddash = 'dash')

fig.show()

## Scatter plot de la moyenne pondérée (y) par année (x) et par genre (catégorie)

In [18]:
fig = px.scatter(data_frame = df_group_years_genres_to_plot, x = "startYear", y = "weighted_rating",
                 color = "genres", color_discrete_sequence = px.colors.qualitative.Light24,
                 width = 1000,
                 labels = {"startYear": "Year", "weighted_rating": "Weighted rating", "genres": "Movie genre"},
                 title = "Ratings of movies produced per year and genre")

fig.update_layout(title = {'y': 0.9, 'x': 0.5, 'xanchor': 'center', 'yanchor': 'top',
                           'font' : dict(size = 24)},
                  plot_bgcolor = 'white')

fig.update_xaxes(ticks = 'outside', showline = True, linecolor = 'black', linewidth = 2,
                 gridcolor = 'lightgrey', griddash = 'dash',
                 range=[1979, 2023])
fig.update_yaxes(ticks = 'outside', showline = True, linecolor = 'black', linewidth = 2,
                 gridcolor = 'lightgrey', griddash = 'dash')

fig.show()

## Line plot de la moyenne pondérée (y) par année (x) et par genre (catégorie)

In [34]:
fig = px.line(data_frame = df_group_years_genres_to_plot, x = "startYear", y = "weighted_rating",
                 color = "genres", color_discrete_sequence = px.colors.qualitative.Light24, markers = True,
                 width = 1000, line_shape='spline',
                 labels = {"startYear": "Year", "weighted_rating": "Weighted rating", "genres": "Movie genre"},
                 title = "Ratings of movies produced per year and genre")

fig.update_layout(title = {'y': 0.9, 'x': 0.5, 'xanchor': 'center', 'yanchor': 'top',
                           'font' : dict(size = 24)},
                  plot_bgcolor = 'white')

fig.update_xaxes(ticks = 'outside', showline = True, linecolor = 'black', linewidth = 2,
                 gridcolor = 'lightgrey', griddash = 'dash',
                 range=[1979, 2023])
fig.update_yaxes(ticks = 'outside', showline = True, linecolor = 'black', linewidth = 2,
                 gridcolor = 'lightgrey', griddash = 'dash')

fig.show()

## Evolution du nombre de films sortis par an et par genre

### Groupement des données par genre et par année et aggrégation

In [32]:
# Création d'un nouveau DataFrame avec uniquement les colonnes concernées ("startYear" et "genres"),
# auxquelles on ajout une colnne "nbMovies".
# Création du DataFrame df_years_genres_movies
df_years_genres_movies = df_movie_title_fr_recent_years_exploded_trim[["startYear", "genres"]]
df_years_genres_movies["nbMovies"] = 1

# Nouveau DataFrame créé par groupement des films par année et par genre
df_years_genres_group = df_years_genres_movies.groupby(by = ["startYear", "genres"]).sum()

# Reset des index du DataFrame df_years_genres_group
df_years_genres_group.reset_index(inplace = True)

df_years_genres_group

C:\Users\julie\AppData\Local\Temp\ipykernel_78744\3995036159.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,startYear,genres,nbMovies
0,1980,Action,104
1,1980,Adventure,60
2,1980,Comedy,188
3,1980,Crime,72
4,1980,Drama,280
...,...,...,...
455,2026,Fantasy,1
456,2026,Romance,1
457,2028,Action,1
458,2028,Adventure,1


### Tracé du line plot du nombre de films produits par an et par genre

In [33]:
fig = px.line(data_frame = df_years_genres_group, x = "startYear", y = "nbMovies",
              color = "genres", color_discrete_sequence = px.colors.qualitative.Light24, markers = True,
              labels = {"startYear": "Year", "nbMovies": "Number of movies", "genres": "Movie genres"},
              title = "Number of movies produced per year and genre", height = 600)

fig.update_layout(title = {'y': 0.9, 'x': 0.5, 'xanchor': 'center', 'yanchor': 'top',
                           'font' : dict(size = 24)}, plot_bgcolor = 'white')

fig.update_xaxes(ticks = 'outside', showline = True, linecolor = 'black', linewidth = 2,
                 gridcolor = 'lightgrey', griddash = 'dash',
                 range = [1979, 2023])
fig.update_yaxes(ticks = 'outside', showline = True, linecolor = 'black', linewidth = 2,
                 gridcolor = 'lightgrey', griddash = 'dash',
                 range = [0, 1100])

fig.show()

# Anciens scripts - ne pas lancer

## Fonction pour lire et traiter le fichier *title_akas*

In [16]:
"""
#@st.cache_data
def load_and_process_title_akas():
    # Définition des colonnes à conserver lors de la lecture du fichier csv
    columns_to_include = ['titleId', 'title', 'region']
    
    # Lecture du fichier csv
    df_title_akas_pyarrow_trim_dtype = pd.read_csv(
        r"https://datasets.imdbws.com/title.akas.tsv.gz", usecols = columns_to_include,
        dtype = {'titleId': str, 'title': str, 'region': str}, delimiter = '\t', engine = "pyarrow")
    
    # Filtre sur la colonne "region" pour ne garder que la valeur "FR"
    df_title_akas_fr = df_title_akas_pyarrow_trim_dtype.loc[df_title_akas_pyarrow_trim_dtype["region"] == "FR"]
    
    # Suppression de la colonne "region"
    df_title_akas_fr_trim = df_title_akas_fr.drop(columns = "region")
    
    return df_title_akas_fr_trim

df_title_akas = load_and_process_title_akas()
df_title_akas.info()
"""

'\n#@st.cache_data\ndef load_and_process_title_akas():\n    # Définition des colonnes à conserver lors de la lecture du fichier csv\n    columns_to_include = [\'titleId\', \'title\', \'region\']\n    \n    # Lecture du fichier csv\n    df_title_akas_pyarrow_trim_dtype = pd.read_csv(\n        r"https://datasets.imdbws.com/title.akas.tsv.gz", usecols = columns_to_include,\n        dtype = {\'titleId\': str, \'title\': str, \'region\': str}, delimiter = \'\t\', engine = "pyarrow")\n    \n    # Filtre sur la colonne "region" pour ne garder que la valeur "FR"\n    df_title_akas_fr = df_title_akas_pyarrow_trim_dtype.loc[df_title_akas_pyarrow_trim_dtype["region"] == "FR"]\n    \n    # Suppression de la colonne "region"\n    df_title_akas_fr_trim = df_title_akas_fr.drop(columns = "region")\n    \n    return df_title_akas_fr_trim\n\ndf_title_akas = load_and_process_title_akas()\ndf_title_akas.info()\n'

## Fonction pour lire et traiter le fichier *title_basics*

In [ ]:
"""
columns_to_include = ['tconst', 'titleType', 'isAdult', 'startYear', 'runtimeMinutes', 'genres']

df_title_basics_pyarrow_trim_dtype = pd.read_csv(
    r"https://datasets.imdbws.com/title.basics.tsv.gz", usecols = columns_to_include,
    dtype = {'tconst': 'string', 'titleType': 'string', 'isAdult': "category",
             'startYear': 'string', 'runtimeMinutes': 'string', 'genres': 'string'},
    delimiter = '\t', low_memory = False)
df_title_basics_pyarrow_trim_dtype.info()
"""

In [ ]:
"""
df_title_basics_recent_years = df_title_basics_pyarrow_trim_dtype.loc[
    df_title_basics_pyarrow_trim_dtype["startYear"].str.isnumeric()]

df_title_basics_recent_years["startYear"] = df_title_basics_recent_years["startYear"].astype("int32")

df_title_basics_recent_years.info()
"""

### Méthode classique

In [ ]:
"""
#@st.cache_data
def load_and_process_title_basics():
    # Définition des colonnes à conserver lors de la lecture du fichier csv
    columns_to_include = ['tconst', 'titleType', 'isAdult', 'startYear', 'runtimeMinutes', 'genres']
    
    # Lecture du fichier csv
    df_title_basics_trim_dtype = pd.read_csv(
        r"https://datasets.imdbws.com/title.basics.tsv.gz", usecols = columns_to_include,
        dtype = {'tconst': 'string', 'titleType': 'string', 'isAdult': "category",
                 'startYear': 'string', 'runtimeMinutes': 'string', 'genres': 'string'},
        delimiter = '\t', low_memory = False)
    
    # Suppression des lignes pour lesquelles la colonne "startYear" ne contient pas de nombre
    df_title_basics_recent_years = df_title_basics_trim_dtype.loc[
        df_title_basics_trim_dtype["startYear"].str.isnumeric()]
    
    # Changement de type de données de la colonne "startYear", passage en type "integer"
    df_title_basics_recent_years["startYear"] = df_title_basics_recent_years["startYear"].astype("int32")
    
    # Filtre sur la colonne "startYear" pour ne garder que les années à partir de 1980
    df_title_basics_recent_years = df_title_basics_recent_years.loc[
        df_title_basics_recent_years["startYear"] >= 1980]
    
    return df_title_basics_recent_years

start_time = time.time()
df_title_basics = load_and_process_title_basics()
end_time = time.time()
loading_time = end_time - start_time
print("Temps de chargement (pandas) :", loading_time, "secondes")

df_title_basics.info()
"""

### Essai avec des "chunks"

In [ ]:
"""
#@st.cache_data
def load_and_process_title_basics_chunks():
    # Définition des colonnes à conserver lors de la lecture du fichier csv
    columns_to_include = ['tconst', 'titleType', 'isAdult', 'startYear', 'runtimeMinutes', 'genres']
    
    df_title_basics_recent_years = pd.DataFrame()
    chunksize = 500000
    
    df_chunks = pd.read_csv(
        r"https://datasets.imdbws.com/title.basics.tsv.gz", usecols = columns_to_include,
        dtype = {'tconst': 'string', 'titleType': 'string', 'isAdult': "category",
                 'startYear': 'string', 'runtimeMinutes': 'string', 'genres': 'string'},
        delimiter = '\t', low_memory = False, chunksize = chunksize)

    for chunk in df_chunks:
        # Filtrer et traiter chaque morceau
        
        # Suppression des lignes pour lesquelles la colonne "startYear" ne contient pas de nombre
        chunk = chunk[chunk["startYear"].str.isnumeric()]
        
        # Changement de type de données de la colonne "startYear", passage en type "integer"
        chunk["startYear"] = chunk["startYear"].astype("int32")
        
        # Filtre sur la colonne "startYear" pour ne garder que les années à partir de 1980
        chunk = chunk[chunk["startYear"] >= 1980]
        
        # Concaténer le morceau traité au DataFrame final
        df_title_basics_recent_years = pd.concat([df_title_basics_recent_years, chunk])
    
    return df_title_basics_recent_years

start_time = time.time()
df_title_basics_chunks = load_and_process_title_basics_chunks()
end_time = time.time()
loading_time = end_time - start_time
print(f"Temps de chargement (avec \"chunks\") : {loading_time} secondes")

df_title_basics_chunks.info()
"""